In [1]:
import mne
import os.path as op

In [5]:
fs_dir = "/home/lauraflyra/Documents/BCCN/NI_project/MNE-fsaverage-data/"
subject_dir = op.dirname(fs_dir)

subject = 'fsaverage'
trans = 'fsaverage'

model = mne.make_bem_model(subject=subject, ico=4, conductivity=(0.3, 0.006, 0.3), subjects_dir=subject_dir)
bem = mne.make_bem_solution(model)

Creating the BEM geometry...
Going from 5th to 4th subdivision of an icosahedron (n_tri: 20480 -> 5120)
Going from 5th to 4th subdivision of an icosahedron (n_tri: 20480 -> 5120)
Going from 5th to 4th subdivision of an icosahedron (n_tri: 20480 -> 5120)
outer skin  CM is  -0.21 -19.38  -0.23 mm
outer skull CM is  -0.19 -19.34  -0.49 mm
inner skull CM is  -0.53 -21.10   6.21 mm
Checking that surface outer skull is inside surface outer skin  ...
Checking that surface inner skull is inside surface outer skull ...
Checking distance between outer skin  and outer skull surfaces...
Minimum distance between the outer skin  and outer skull surfaces is approximately    1.6 mm
Checking distance between outer skull and inner skull surfaces...
Minimum distance between the outer skull and inner skull surfaces is approximately    5.4 mm
Surfaces passed the basic topology checks.
Complete.

Approximation method : Linear collocation

Three-layer model surfaces loaded.
Computing the linear collocation s

In [6]:
src = mne.setup_volume_source_space(subject=subject, bem=bem, subjects_dir=subject_dir, add_interpolator=False)

BEM              : <ConductorModel | BEM (3 layers)>
grid                  : 5.0 mm
mindist               : 5.0 mm
MRI volume            : /home/lauraflyra/Documents/BCCN/NI_project/MNE-fsaverage-data/fsaverage/mri/T1.mgz

Reading /home/lauraflyra/Documents/BCCN/NI_project/MNE-fsaverage-data/fsaverage/mri/T1.mgz...

Taking inner skull from <ConductorModel | BEM (3 layers)>
Surface CM = (  -0.5  -21.1    6.2) mm
Surface fits inside a sphere with radius   98.3 mm
Surface extent:
    x =  -75.2 ...   76.3 mm
    y = -113.3 ...   75.0 mm
    z =  -71.7 ...   88.2 mm
Grid extent:
    x =  -80.0 ...   80.0 mm
    y = -115.0 ...   75.0 mm
    z =  -75.0 ...   90.0 mm
43758 sources before omitting any.
29824 sources after omitting infeasible sources not within 0.0 - 98.3 mm.
Source spaces are in MRI coordinates.
Checking that the sources are inside the surface and at least    5.0 mm away (will take a few...)
Checking surface interior status for 29824 points...
    Found  3653/29824 points insi